What I need to do here to make sure that there is no confusion about the positions of opposite homozygotes is to call SNPs using the joint calling with the 2K matrix.

In [15]:
library(readxl)

source("../../BrusselSprouts/scripts/functions.R")
crosses=list.files("~/data/trd/mapped_reads",pattern = "g.vcf.gz$",full.names = TRUE)

my_crosses=read_xlsx("~/data/trd/Crosses.xlsx", sheet=2)[,c("Short name 1", "Short name 2", "Status", "Colonies in pool")]
chris_crosses=data.frame(sample1=c("ACP","BAP","CCD","ATE","ACK","AKE","BAH","ANG"),
                        sample2=c("BFP","CMP","CPG","YCR","ACV","BAH","CGD","CEI"))
all_used_samples=unique(c(my_crosses$`Short name 1`, my_crosses$`Short name 2`, chris_crosses$sample1, chris_crosses$sample2))
all_used_samples[all_used_samples=="YCR"]=paste0("SACE_",all_used_samples[all_used_samples=="YCR"])
summary(file.exists(paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz")))

all_used_samples[!file.exists(paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz"))]


samples=c(crosses,paste0("/home/jnrunge/data/trd/GVCF_2489Strains/",all_used_samples,".g.vcf.gz"))
samples=data.frame(id=unlist(lapply(basename(samples), getFirst_v2, split=".")),file=samples)
fwrite(samples, "~/data/trd/mapped_reads/all_samples.tsv", col.names = FALSE, sep="\t")
head(samples)

New names:
* `Is in Stock` -> `Is in Stock...4`
* `Is in Stock` -> `Is in Stock...6`


   Mode    TRUE 
logical      57 

character(0)

,id,file
,<chr>,<chr>
1,YJNRC2,/home/jnrunge/data/trd/mapped_reads/YJNRC2.g.vcf.gz
2,BAK,/home/jnrunge/data/trd/GVCF_2489Strains/BAK.g.vcf.gz
3,BAN,/home/jnrunge/data/trd/GVCF_2489Strains/BAN.g.vcf.gz
4,AKH,/home/jnrunge/data/trd/GVCF_2489Strains/AKH.g.vcf.gz
5,ABS,/home/jnrunge/data/trd/GVCF_2489Strains/ABS.g.vcf.gz
6,AMM,/home/jnrunge/data/trd/GVCF_2489Strains/AMM.g.vcf.gz


In [9]:
# make BED file (all positions because GATK is retarded)
fai=fread("~/data/TRD/R64_nucl.fasta.fai")
bed=data.frame(chr=fai$V1, from=1, to=fai$V2)
head(bed)
fwrite(bed, "~/data/TRD/R64_nucl.fasta.fai.bed", col.names = FALSE, sep="\t")

,chr,from,to
,<chr>,<dbl>,<int>
1,chromosome1,1,230218
2,chromosome2,1,813184
3,chromosome3,1,316620
4,chromosome4,1,1531933
5,chromosome5,1,576874
6,chromosome6,1,270161


In [20]:
scripts_dir="/home/jnrunge/data/trd/mapped_reads/scripts/"

In [27]:
initial_timedate=Sys.time()

# what needs to be done is individually, and parallely genotype all crosses, then merge them into a DB, and once call the joint genotyping (once per batch of sequencing, so twice in total)

bam_files=list.files(path="~/data/trd/mapped_reads",pattern="JNRC[0-9]*.bam$",full.names = TRUE)
bam_files=naturalsort(bam_files)
crosses=unlist(lapply(basename(bam_files), getFirst_v2, split="."))
for(cross in crosses){
    print(cross)

    if(file.exists(paste0("/home/jnrunge/data/trd/mapped_reads/",cross,".g.vcf.gz"))){
        print("GVCF exists")
        next
    }

    cmds=paste0("cd /home/jnrunge/data/trd/mapped_reads/ && gatk AddOrReplaceReadGroups -I ",cross,".bam -O ",cross,".ReadGroups.bam --RGID ",cross," --RGLB ",cross," --RGPL ILLUMINA --RGPU ",cross," --RGSM ",cross,"")
    cmds=c(cmds,paste0("mv -f ",cross,".ReadGroups.bam ",cross,".bam"))
    cmds=c(cmds,paste0("samtools index ",cross,".bam"))
    cmds=c(cmds,paste0("gatk HaplotypeCaller -R ../../TRD/R64_nucl.fasta -I ",cross,".bam -O ",cross,".g.vcf.gz --emit-ref-confidence GVCF"))

    sbatch_list=execute_complex_sbatch(cmds, jobname = jobname<-"GATK", scripts_dir = scripts_dir, uniqueRunID = cross, cores="1", mem="8gb", time="short", env="bwaetc", initial_timedate = initial_timedate, jobs_simul = 10, jobs_total = 10)


}

if(exists("sbatch_list")){
    start_sbatch_list(sbatch_list, 10, jobname, initial_timedate)
    }

[1] "YJNRC2"
[1] "GVCF exists"
[1] "YJNRC17"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/GATK-YJNRC17.sbatch"
[1] "YJNRC18"
[1] "/home/jnrunge/data/trd/mapped_reads/scripts/GATK-YJNRC18.sbatch"
[1] "2 /home/jnrunge/data/trd/mapped_reads/scripts/GATK.list"
[1] "[1] \"Currently running 0 jobs.\""               
[2] "[1] \"2 sbatches remaining! Running 11 more...\""
[3] "[1] \"Submitted batch job 462634\""              
[4] "[1] \"Submitted batch job 462635\""              


running C2 in int1

````bash

gatk GenomicsDBImport --batch-size 200 --genomicsdb-workspace-path TRD_DB --sample-name-map ~/data/trd/mapped_reads/all_samples.tsv -L ~/data/TRD/R64_nucl.fasta.fai.bed

gatk GenotypeGVCFs -R ../../TRD/R64_nucl.fasta -V gendb://TRD_DB -O TRD.vcf.gz


````